In [1]:
import numpy as np
from treys import Card, Evaluator
from pokerenv.common import GameState, PlayerState, PlayerAction, Action
from pokerenv.player import Player
from MODIFIEDTABLE import ModifiedTable  # Adjust import path as needed


In [2]:
import random
from pokerenv.common import PlayerAction, Action

def realistic_debug_simulation(table):
    """
    Simulate a realistic poker round with predefined small blind, big blind, and logical player actions.
    
    Args:
        table: An instance of ModifiedTable.
    """
    print("---- Starting Realistic Debug Simulation ----")

    # Reset the table
    obs = table.reset()

    # Set small blind and big blind actions
    sb_position = (table.current_dealer + 1) % table.n_players
    bb_position = (sb_position + 1) % table.n_players

    # Mandatory small blind and big blind actions
    print(f"\nSmall Blind: {table.players[sb_position].name} posts ${table.SB}")
    table.step(Action(action_type=PlayerAction.BET, bet_amount=table.SB))
    
    print(f"\nBig Blind: {table.players[bb_position].name} posts ${table.BB}")
    table.step(Action(action_type=PlayerAction.BET, bet_amount=table.BB))

    # Player actions
    done = False
    while not done:
        current_player = table.players[table.next_player_i]
        # Skip folded or all-in players
        if current_player.state != PlayerState.ACTIVE or current_player.all_in:
            continue

        # Make a decision for the current player
        valid_actions = table._get_valid_actions(current_player)
        action = None

        # Simple strategy: 60% chance to call, 20% to fold, 20% to raise
        decision = random.random()
        if decision < 0.6 and PlayerAction.CALL in valid_actions:
            action = Action(action_type=PlayerAction.CALL)
            print(f"{current_player.name}: calls.")
        elif decision < 0.8 and PlayerAction.FOLD in valid_actions:
            action = Action(action_type=PlayerAction.FOLD)
            print(f"{current_player.name}: folds.")
        elif PlayerAction.BET in valid_actions:
            raise_amount = random.randint(table.BB, current_player.stack // 2)
            action = Action(action_type=PlayerAction.BET, bet_amount=raise_amount)
            print(f"{current_player.name}: raises to ${raise_amount}.")
        else:
            print(f"{current_player.name}: checks.")
            action = Action(action_type=PlayerAction.CHECK)

        # Step the environment with the chosen action
        obs, reward, done, _ = table.step(action)
        # End the loop if the hand is over
        if done:
            print("\nThe hand has ended.")
            break

    # Final pot distribution and debug
    table._distribute_pot()
    print("---- Realistic Debug Simulation Complete ----")

# Example usage
table = ModifiedTable(
    n_players=6,
    stack_low=100,
    stack_high=101,
    SB=5,
    BB=10
)
realistic_debug_simulation(table)


---- Starting Realistic Debug Simulation ----

Small Blind: player_3 posts $5
---- Debugging Pot Distribution ----
Total Pot: $15.00
Player Contributions to Pot:
  player_1: Money in Pot: $0.00, Stack: $100.00
  player_2: Money in Pot: $0.00, Stack: $100.00
  player_3: Money in Pot: $5.00, Stack: $95.00
  player_4: Money in Pot: $10.00, Stack: $90.00
  player_5: Money in Pot: $0.00, Stack: $100.00
  player_6: Money in Pot: $0.00, Stack: $100.00
Pot matches total contributions.
---- End Debugging Pot Distribution ----
---- Debugging Pot Distribution ----
Total Pot: $15.00
Player Contributions to Pot:
  player_1: Money in Pot: $0.00, Stack: $100.00
  player_2: Money in Pot: $0.00, Stack: $100.00
  player_3: Money in Pot: $5.00, Stack: $95.00
  player_4: Money in Pot: $10.00, Stack: $90.00
  player_5: Money in Pot: $0.00, Stack: $100.00
  player_6: Money in Pot: $0.00, Stack: $100.00
Pot matches total contributions.
---- End Debugging Pot Distribution ----

Big Blind: player_4 posts $10
-

In [3]:
def test_split_pot():
    """Test case for equal hands and split pot scenarios"""
    table = ModifiedTable(n_players=6, 
                         stack_low=5000,  
                         stack_high=5001,  
                         SB=5, 
                         BB=10)
    table.hand_history_enabled = True

    def test_equal_hands_split():
        table.reset()
        
        # Force specific cards for players and board
        table.players[0].cards = [Card.new('As'), Card.new('Kh')]  # Player 1
        table.players[1].cards = [Card.new('Ac'), Card.new('Qh')]  # Player 2
        table.cards = [
            Card.new('Ah'),
            Card.new('2c'),
            Card.new('3d'),
            Card.new('4h'),
            Card.new('5s')
        ]
        
        # Simulate betting
        p1_bet = 1000
        p2_bet = 1000
        table.pot = p1_bet + p2_bet
        table.players[0].money_in_pot = p1_bet
        table.players[1].money_in_pot = p2_bet
        table.players[0].stack -= p1_bet
        table.players[1].stack -= p2_bet
        
        # Set players as active and others as folded
        for i, player in enumerate(table.players):
            if i < 2:
                player.state = PlayerState.ACTIVE
            else:
                player.state = PlayerState.FOLDED
        table.active_players = 2
        
        table.street = GameState.RIVER
        
        # Distribute pot
        table._distribute_pot()
        
        # Check results
        assert abs(table.players[0].winnings - table.players[1].winnings) < 0.01, \
            f"Players should have equal winnings, got {table.players[0].winnings} vs {table.players[1].winnings}"
        assert abs(table.players[0].winnings - 1000) < 0.01, \
            f"Each player should win 1000, got {table.players[0].winnings}"

    def test_all_in_side_pots():
        table.reset()
        
        print("\n=== Test Setup ===")
        # Set up players with their raw amounts
        setup = [
            (500, ['As', 'Ks']),   # Player 0: Best hand
            (1000, ['Kh', 'Qh']),  # Player 1: Second best
            (1000, ['Jh', 'Th']),  # Player 2: Worst hand
        ]
        
        # Set up the players
        for i, (stack, cards) in enumerate(setup):
            table.players[i].stack = stack
            table.players[i].cards = [Card.new(card) for card in cards]
            table.players[i].money_in_pot = stack  # Just use raw value
            table.players[i].state = PlayerState.ACTIVE
            table.players[i].all_in = True
            print(f"Setting up {table.players[i].name}:")
            print(f"- Stack: {stack}")
            print(f"- Cards: {cards}")
        
        # Set up board
        board_cards = ['Ac', '2h', '4c', '7s', 'Kc']
        table.cards = [Card.new(card) for card in board_cards]
        print(f"\nBoard: {board_cards}")
        
        table.street = GameState.RIVER
        table.pot = sum(p.money_in_pot for p in table.players[:3])
        table.active_players = 3
        
        # Set remaining players as folded
        for i in range(3, len(table.players)):
            table.players[i].state = PlayerState.FOLDED
            table.players[i].money_in_pot = 0
        
        # Print expected pot distribution
        print("\n=== Expected Distribution ===")
        print(f"Main pot (all players): 1500")  # 3 * 500
        print(f"Side pot 1 (P1 & P2): 1000")    # 2 * 500
        print(f"Side pot 2 (P2 only): 1000")    # 1 * 1000
        
        # Distribute pot
        table._distribute_pot()
        
        # Verify correct distribution
        main_pot = 1500   # 3 players × 500
        side_pot1 = 1000  # 2 players × 500
        
        print("\n=== Final Verification ===")
        print(f"P0 expected: {main_pot}, actual: {table.players[0].winnings}")
        print(f"P1 expected: {side_pot1}, actual: {table.players[1].winnings}")
        print(f"P2 expected: 0, actual: {table.players[2].winnings}")
        
        # Assertions
        assert abs(table.players[0].winnings - main_pot) < 0.01, \
            f"Player 0 should win main pot ({main_pot}), got {table.players[0].winnings}"
        assert abs(table.players[1].winnings - side_pot1) < 0.01, \
            f"Player 1 should win first side pot ({side_pot1}), got {table.players[1].winnings}"
        assert abs(table.players[2].winnings) < 0.01, \
            f"Player 2 should win nothing, got {table.players[2].winnings}"
    
    test_all_in_side_pots()
    print("All tests passed!")

In [4]:
test_split_pot()


=== Test Setup ===
Setting up player_1:
- Stack: 500
- Cards: ['As', 'Ks']
Setting up player_2:
- Stack: 1000
- Cards: ['Kh', 'Qh']
Setting up player_3:
- Stack: 1000
- Cards: ['Jh', 'Th']

Board: ['Ac', '2h', '4c', '7s', 'Kc']

=== Expected Distribution ===
Main pot (all players): 1500
Side pot 1 (P1 & P2): 1000
Side pot 2 (P2 only): 1000

=== Final Verification ===
P0 expected: 1500, actual: 1000.0
P1 expected: 1000, actual: 0.0
P2 expected: 0, actual: -1000


AssertionError: Player 0 should win main pot (1500), got 1000.0

In [ ]:
print(table.)